In [4]:
'''
    プロトタイピング用のパスと、Botライブラリーパスを取得／設定します
'''
import sys
import os

prototype_dir = os.path.join(os.getcwd(), '..')
prototype_dir = os.path.abspath(prototype_dir)

learning_dir = os.path.join(prototype_dir, '..')
learning_dir = os.path.abspath(learning_dir)
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

print('prototype_dir=%s\nlearning_dir=%s' % (prototype_dir, learning_dir))

prototype_dir=/Users/harada/source_code/donusagi-bot/learning/prototype
learning_dir=/Users/harada/source_code/donusagi-bot/learning


In [33]:
import pandas as pd
from IPython.display import display, HTML

feedbacks =  pd.DataFrame({
    'id': [13, 1, 2],
    'question': [
        'ピアノの先生を見つけたい',
        'ダミーのデータ1',
        'ダミーのデータ2',
    ],
})
display(feedbacks)

,id,question
0,13,ピアノの先生を見つけたい
1,1,ダミーのデータ1
2,2,ダミーのデータ2


In [11]:
question_answers =  pd.DataFrame({
    'id': [11,12,13],
    'question': [
        'ピアノ持ってない',
        'うちの子供は2歳です。ピアノを始めるにはまだ早いでしょうか？ ',
        '国立音大の先生を探しているのですが。'
    ],
    'answer': [
        'お申込時点でピアノをご家庭にお持ちでなくても、基本は問題ありません。',
        '一般に、ピアノを習うのは早ければ早いほどいい、と言われます。',
        '音大勤務の指定については、お申込時にご要望欄へ記入ください。個別にご相談に応じます。'
    ]
})
display(question_answers)

,answer,id,question
0,お申込時点でピアノをご家庭にお持ちでなくても、基本は問題ありません。,11,ピアノ持ってない
1,一般に、ピアノを習うのは早ければ早いほどいい、と言われます。,12,うちの子供は2歳です。ピアノを始めるにはまだ早いでしょうか？
2,音大勤務の指定については、お申込時にご要望欄へ記入ください。個別にご相談に応じます。,13,国立音大の先生を探しているのですが。


In [29]:
# 普通のコサイン類似検索をする
from learning.core.training_set.text_array import TextArray
from sklearn.metrics.pairwise import cosine_similarity

questions = question_answers['question']
text_array = TextArray(questions)
print(text_array.to_vec().toarray().shape)

X = ['ピアノの先生を見つけたいです']
q_text_array = TextArray(X, vectorizer=text_array.vectorizer)
print(q_text_array.to_vec().toarray().shape)

similarities = cosine_similarity(text_array.to_vec(), q_text_array.to_vec())

# question_answersのid:13のデータが類似度が高いという結果が出る
print(similarities)

2017/09/23 PM 05:41:52 TextArray#__init__ start
2017/09/23 PM 05:41:52 TextArray#to_vec start
2017/09/23 PM 05:41:52 TextArray#to_vec end
2017/09/23 PM 05:41:52 TextArray#__init__ start
2017/09/23 PM 05:41:52 TextArray#to_vec start
2017/09/23 PM 05:41:52 TextArray#to_vec end
2017/09/23 PM 05:41:52 TextArray#to_vec start
2017/09/23 PM 05:41:52 TextArray#to_vec end
2017/09/23 PM 05:41:52 TextArray#to_vec start
2017/09/23 PM 05:41:52 TextArray#to_vec end


(3, 13)
(1, 13)
[[ 0.28671097]
 [ 0.16723635]
 [ 0.45954803]]


In [54]:
# feedbacksからidをコサイン類似検索する
questions = feedbacks['question']
text_array = TextArray(questions)
print(text_array.to_vec().toarray().shape)

X = ['ピアノの先生を見つけたいです']
q_text_array = TextArray(X, vectorizer=text_array.vectorizer)
print(q_text_array.to_vec().toarray().shape)

similarities = cosine_similarity(text_array.to_vec(), q_text_array.to_vec())
similarities = similarities.flatten()
feedbacks['similarity'] = similarities

# 最もsimilarityが高いidを取得する
display(feedbacks)
feedbacks = feedbacks.sort_values(by='similarity', ascending=False)
display(feedbacks[:1])
predicted_id = feedbacks[:1]['id']

2017/09/23 PM 05:58:46 TextArray#__init__ start
2017/09/23 PM 05:58:46 TextArray#to_vec start
2017/09/23 PM 05:58:46 TextArray#to_vec end
2017/09/23 PM 05:58:46 TextArray#__init__ start
2017/09/23 PM 05:58:46 TextArray#to_vec start
2017/09/23 PM 05:58:46 TextArray#to_vec end
2017/09/23 PM 05:58:46 TextArray#to_vec start
2017/09/23 PM 05:58:46 TextArray#to_vec end
2017/09/23 PM 05:58:46 TextArray#to_vec start
2017/09/23 PM 05:58:46 TextArray#to_vec end


(3, 7)
(1, 7)


,id,question,similarity
0,13,ピアノの先生を見つけたい,1.0
1,1,ダミーのデータ1,0.0
2,2,ダミーのデータ2,0.0


,id,question,similarity
0,13,ピアノの先生を見つけたい,1.0


In [60]:
# idを加えて普通のコサイン類似検索をする
from learning.core.training_set.text_array import TextArray
from sklearn.metrics.pairwise import cosine_similarity

questions = question_answers['question']
questions = question_answers[['question', 'id']].apply(lambda x: '{} {}'.format(x[0], x[1]), axis=1)
display(questions)

text_array = TextArray(questions)
print(text_array.to_vec().toarray().shape)

X_new = [X[0] + ' ' + str(predicted_id)]
q_text_array = TextArray(X_new, vectorizer=text_array.vectorizer)
print(q_text_array.to_vec().toarray().shape)

# idを加えてコサイン類似検索することで、正解データのｓimilarityが  0.45954803 => 0.62276601 に向上した
similarities = cosine_similarity(text_array.to_vec(), q_text_array.to_vec())
display(similarities)

0                           ピアノ持ってない 11
1    うちの子供は2歳です。ピアノを始めるにはまだ早いでしょうか？  12
2                 国立音大の先生を探しているのですが。 13
dtype: object

2017/09/23 PM 05:59:36 TextArray#__init__ start
2017/09/23 PM 05:59:36 TextArray#to_vec start
2017/09/23 PM 05:59:36 TextArray#to_vec end
2017/09/23 PM 05:59:36 TextArray#__init__ start
2017/09/23 PM 05:59:36 TextArray#to_vec start
2017/09/23 PM 05:59:36 TextArray#to_vec end
2017/09/23 PM 05:59:36 TextArray#to_vec start
2017/09/23 PM 05:59:36 TextArray#to_vec end
2017/09/23 PM 05:59:36 TextArray#to_vec start
2017/09/23 PM 05:59:36 TextArray#to_vec end


(3, 16)
(1, 16)


array([[ 0.19041816],
       [ 0.12298429],
       [ 0.62276601]])

In [ ]:
# TODO: Badフィードバックのケース